# [Day 8](https://adventofcode.com/2022/day/8)

## Read input file

In [1]:
import numpy as np
import pandas as pd

trees = list()
with open('input.txt', 'r') as file:
    while line := [int(x) for x in [*file.readline().strip()]]:
        trees.append(line)
trees = np.array(trees, dtype=int)

## Part 1

In [2]:
total = 0

def count_row(trees, row, mask):
    count = 0
    
    taller = -1    
    it = range(0, len(trees[row]))
    for col in it:
        if trees[row][col] > taller:
            count += 1 if mask[row][col] else 0
            taller = trees[row][col]
            mask[row][col] = 0
            
    taller = -1
    for col in reversed(it):
        if trees[row][col] > taller:
            count += 1 if mask[row][col] else 0
            taller = trees[row][col]
            mask[row][col] = 0

    return (count, mask)

mask = np.ones((len(trees),len(trees[0])))            
for row in range(len(trees)):
    c, mask = count_row(trees, row, mask)
    total += c

mask = mask.T

for row in range(len(trees.T)):
    c, mask = count_row(trees.T, row, mask)
    total += c

f"Total: {total}"

'Total: 1792'

## Part 2

In [3]:
# not in the mood to do an efficient algorithm, will use brute force 
def best_scenic(trees):
    best = 0
    views = np.zeros((len(trees), len(trees[0])))
    for row in range(1, len(trees)-1):
        for col in range(1, len(trees[row])-1):
            best = max(best, scenic(trees, row, col))
            
    return best
            
def scenic(trees, row, col):
    total_scenic = 1
    
    count = 1
    for r in range(row-1, 0, -1):
        if trees[row][col] <= trees[r][col]:
                break
        count += 1
    total_scenic *= count
    
    count = 1
    for r in range(row+1, len(trees)-1):
        if trees[row][col] <= trees[r][col]:
            break
        count += 1
    total_scenic *= count
    
    count = 1
    for c in range(col-1, 0, -1):
        if trees[row][col] <= trees[row][c]:
                break
        count += 1
    total_scenic *= count
    
    count = 1
    for c in range(col+1, len(trees[row])-1):
        if trees[row][col] <= trees[row][c]:
            break
        count += 1
    total_scenic *= count

    return total_scenic

f"Best: {best_scenic(trees)}"

'Best: 334880'

# DAG solution

In [4]:
def left_dist(row):
    left = [0 for _ in row]
    m = 0
    for i, height in enumerate(row):
        if i > 0 and height <= row[i-1]:
            m = i-1
        left[i] = m
    distances = [0 for _ in row]
    for i, height in enumerate(row):
        idx = i
        dist = idx - left[idx]
        idx = left[idx]
        while idx > 0 and height > row[idx]:
            dist += idx - left[idx]
            left[i] = idx
            idx = left[idx]
        distances[i] = dist
    return distances

def row_dist(row):
    left = left_dist(row)
    right = np.flip(left_dist(np.flip(row)))
    return np.multiply(left,right)

def trees_dist(trees):
    tree_dist = np.ones(trees.shape,dtype=int)
    for i, row in enumerate(trees):
        np.multiply(tree_dist[i], row_dist(row), tree_dist[i])
    for i, row in enumerate(trees.T):
        np.multiply(tree_dist.T[i], row_dist(row), tree_dist.T[i])
    return tree_dist
distances = trees_dist(trees);
display(pd.DataFrame(distances))
f"Best: {np.max(distances)}"

,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,96,97,98
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,42,1,10,1,1,1,1,84,1,...,4,6,1,4,1,4,10,1,2,0
2,0,1,8,216,1,1,18,1,1,48,...,1,1,14,1,12,1,1,12,1,0
3,0,1,1,1,24,15,216,6,2,4,...,60,2,90,4,1,216,1,1,24,0
4,0,3,2,4,1,400,1,2,15,288,...,1,36,2,1,60,2,3,16,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0,2,8,72,36,4,2,24,3,24,...,1,1,18,1,1,7,32,9,64,0
95,0,3,1,1,36,2,1,21,40,3,...,24,6,1,4,24,12,4,1,6,0
96,0,2,1,16,1,2,24,2,1,8,...,2,2,3,28,1,8,3,1,1,0
97,0,30,1,1,6,1,1,6,16,3,...,1,1,84,1,4,1,2,12,1,0


'Best: 334880'